# <span style="color:orange">Nettoyer des jeux de données pour obtenir une liste de DOI des publications de l'Inserm : Web of Science, Pubmed, HAL</span>

Ce premier notebook sert à nettoyer les différents fichiers obtenus après téléchargement sur le WoS, Pubmed, HAL. Pour savoir quelques requêtes ont été utilisées pour l'Inserm, consulter le fichier "requetes_bdd" dans le dossier. Quelques consignes sont à respecter pour que tout fonctionne :

- Pour le WoS, il suffit de procéder à un téléchargement simple "Fast 5000". Le fichier obtenu, en texte, est illisible et c'est normal, il n'y a rien à changer. Nommer le fichier "wos_lorraine_2016", puis "wos_lorraine_2017"... Ce fichier n'apparaît pas dans le dossier téléchargé depuis Gitlab car les données du Web of Science étant propriétaires, il n'était pas possible de les diffuser librement.
- Pour Pubmed, le téléchargement donne un fichier CSV très peu classé, c'est normal, il n'y a rien à changer. Nommer le fichier "pubmed_inserm_2016", puis "pubmed_inserm_2017"...
- Pour les autres sources de données, on obtient directement une liste de DOI, mais **il faut s'assurer que la colonne s'appelle bien "doi" en minuscules et qu'il n'y a pas de ligne vide**

Il faut télécharger année par année, et toujours nommer les fichiers de la même manière.  <span style="color:red"> **Il est vital de garder l'organisation ici présente (Data > raw > dossier par année) pour que le code fonctionne.**</span>

<span style="color:red">**Si l'on ne dispose pas de certaines données (par exemple, l'établissement n'a pas de données sur les APC), il faut quand même créer un fichier Excel avec une colonne "doi", et le laisser vide. Sans cela, le code ne fonctionnera pas.**</span>

In [1]:
column_name = "doi"

## <span style="color:orange">Nettoyer les données issues du Web of Science</span>

### Comprendre comment est structuré le fichier

On ouvre le premier fichier puis on utilise la méthode de liste **split** pour voir une ligne (la 4ème, par exemple).

In [2]:
with open("Data/raw/2016/wos_inserm_2016.txt") as f:
    wos_string = f.read()

La ligne ci-dessous permet de voir à quoi ressemble la 5ème ligne du fichier (en Python, la numérotation commence à zéro).

In [3]:
wos_string.split("\n") [4]

'J\tPavasini, Rita; Guralnik, Jack; Brown, Justin C.; di Bari, Mauro; Cesari, Matteo; Landi, Francesco; Vaes, Bert; Legrand, Delphine; Verghese, Joe; Wang, Cuiling; Stenholm, Sari; Ferrucci, Luigi; Lai, Jennifer C.; Arnau Bartes, Anna; Espaulella, Joan; Ferrer, Montserrat; Lim, Jae-Young; Ensrud, Kristine E.; Cawthon, Peggy; Turusheva, Anna; Frolova, Elena; Rolland, Yves; Lauwers, Valerie; Corsonello, Andrea; Kirk, Gregory D.; Ferrari, Roberto; Volpato, Stefano; Campo, Gianluca\t\t\t\t\t\t\t\tShort Physical Performance Battery and all-cause mortality: systematic review and meta-analysis\t\t\t\t\t\t\t\tBMC MEDICINE\t\t\t\t14\t\t\t\t\t\t215\t10.1186/s12916-016-0763-7\t\t\tDEC 22 2016\t2016\tBackground: The Short Physical Performance Battery (SPPB) is a well-established tool to assess lower extremity physical performance status. Its predictive ability for all-cause mortality has been sparsely reported, but with conflicting results in different subsets of participants. The aim of this stud

### Importer la librairie **Pandas** et définir une fonction

La librairie Pandas va permettre de manipuler les fichiers.

In [4]:
import pandas

In [5]:
import csv

Définition de la fonction **read_wos** : l'année est en argument. On tape la 1ère année concernée (2016) pour que cela utilise les données de 2016 en premier. Dans cette fonction, les fichiers de toutes les années (de 2016 à 2021 exclue) seront interrogés. 

On crée une liste vide (wos_per_year) dans laquelle on stockera après les données de toutes les années grâce à la boucle **for**.  <span style="color:red">**Pour l'instant, on n'interroge que jusqu'en 2020. Il faudra mettre 2022 à la place quand les données de 2021 seront ajoutées dans le dossier "raw", et ainsi de suite.**</span>

La liste vide wos_per_year est remplie à chaque tour de boucle grâce à la méthode **append**. Pour avoir la liste de toutes les données sans tri par année, on concatène les fichiers grâce à **pandas.concat**.

In [6]:
# Une année de début peut etre fournie, sinon par défaut commence en 2016
def read_wos(year=2016):
    wos_per_year = []
    for y in range (year, 2021):
       wos_df = pandas.read_csv("Data/raw/{}/wos_inserm_{}.txt".format(str(y), str(y)), sep="\t",
                 quoting=csv.QUOTE_NONE, index_col=False, usecols=['DI'])
       wos_per_year.append(wos_df)

    full_wos = pandas.concat(wos_per_year)
    
    resultat = full_wos.rename (columns={'DI':column_name})
           
    return resultat

In [7]:
wos_df = read_wos()

In [8]:
wos_df

,doi
0,10.1186/s13601-016-0137-4
1,10.1371/journal.pone.0168349
2,10.3389/fnmol.2016.00160
3,10.1186/s12916-016-0763-7
4,10.3389/fimmu.2016.00632
...,...
1908,10.1038/s41586-020-2338-1
1909,10.1038/s41586-020-1969-6
1910,10.1016/S0140-6736(20)30752-2
1911,NaN


Vérifier que le chiffre total obtenu ici correspond bien au chiffre trouvé dans le Web of Science pour toutes les années. Ne pas hésiter à refaire les extractions d'années précédentes si les chiffres ne correspondent pas : il peut y avoir de nouvelles publications indexées dans le Web of Science.

### Identifier les cellules qui n'ont pas de DOI

Trouver le nombre de lignes qui n'ont pas de DOI : utiliser la méthode **is not a number** (isna).

In [9]:
def wos_sans_doi():
    # wos_df.isna().sum()
    return wos_df.isna().sum()

On met [0] pour demander le 1er élément de la liste

In [10]:
wos_sans_doi()[0]

845

Trouver le nombre de lignes qui ont un DOI : utiliser la méthode **match** sur le string.

In [11]:
def wos_avec_doi():
    # wos_df[column_name].str.match("10.").sum()
    return wos_df[column_name].str.match("10.").sum()

In [12]:
wos_avec_doi()

60581

Vérifier le nombre de lignes total :

In [13]:
len(wos_df)

61426

Vérifier qu'on a bien tout récupéré :

In [14]:
wos_sans_doi() + wos_avec_doi()

doi    61426
dtype: int64

Méthode de vérification de Python : **assert**. Si cela ne renvoie rien, c'est que les deux valeurs comparées sont identiques.

In [15]:
expected = len(wos_df[column_name])
actual = wos_sans_doi()[0] + wos_avec_doi()

assert(expected == actual)

 <span style="color:green"> **Les données du WoS sont à présent nettoyées et stockées dans la variable intitulée "wos_df".** </span>

## <span style="color:orange">Nettoyer les données issues de Pubmed</span>

In [16]:
with open("Data/raw/2016/pubmed_inserm_2016.csv", encoding='utf-8') as f:
    pubmed_string = f.read()

In [17]:
pubmed_string.split("\n")[1]

'"26967288","Communication by Extracellular Vesicles: Where We Are and Where We Need to Go","Tkach M, Théry C.","Cell. 2016 Mar 10;164(6):1226-1232. doi: 10.1016/j.cell.2016.01.043.","Tkach M","Cell","2016","2016/03/12","","","10.1016/j.cell.2016.01.043"'

In [18]:
pubmed_df = pandas.read_csv("Data/raw/2020/pubmed_inserm_2020.csv", encoding='utf-8')

In [19]:
print(pubmed_df.columns)

Index(['PMID', 'Title', 'Authors', 'Citation', 'First Author', 'Journal/Book',
       'Publication Year', 'Create Date', 'PMCID', 'NIHMS ID', 'DOI'],
      dtype='object')


Il y a bien une colonne qui s'appelle "DOI", elle va être interrogée pour trouver directement les DOI. <span style="color:red">**Pour l'instant, on n'interroge que jusqu'en 2020. Il faudra mettre 2022 à la place quand les données de 2021 seront ajoutées dans le dossier "raw", et ainsi de suite.**</span>


In [20]:
# Une année de début peut etre fournie, sinon par défaut commence en 2016
def read_pubmed(year=2016):
    pubmed_per_year = []
    for y in range (year, 2021):
        pubmed_df = pandas.read_csv("Data/raw/{}/pubmed_inserm_{}.csv".format(str(y), str(y))
                                    ,sep=",", encoding='utf-8')
        pubmed_per_year.append(pubmed_df)
    
    full_pubmed = pandas.concat(pubmed_per_year)
           
    resultat = pandas.DataFrame({column_name: full_pubmed["DOI"]})                      
    return resultat

In [21]:
pubmed_df = read_pubmed()

### Vérifier qu'il ne manque pas de DOI

In [22]:
pubmed_df

,doi
0,10.1016/j.cell.2016.01.043
1,10.1002/wdev.201
2,10.1016/j.cmi.2016.08.031
3,10.1016/S0151-9638(18)30079-6
4,10.1016/j.jsbmb.2015.09.030
...,...
24669,10.1111/all.14471
24670,10.1038/s41436-020-0862-x
24671,10.1038/s41588-020-00713-x
24672,10.1007/s00705-020-04731-2


In [23]:
def pubmed_sans_doi():
    # pubmed_df.isna().sum()
    return pubmed_df.isna().sum()

In [24]:
pubmed_sans_doi()

doi    475
dtype: int64

In [25]:
def pubmed_avec_doi():
    # pubmed_df[column_name].str.match("10.").sum()
    return pubmed_df[column_name].str.match("10.").sum()

In [26]:
pubmed_avec_doi()

64199

In [27]:
expected = len(pubmed_df)
actual = pubmed_sans_doi()[0] + pubmed_avec_doi()

assert(expected == actual)

 <span style="color:green"> **Les données de Pubmed sont à présent nettoyées et stockées dans la variable intitulée "pubmed_df".** </span>

## <span style="color:orange">Ajouter les données de HAL</span>

Grâce aux API, on peut récupérer une grande liste de DOI. Il peut être intéressant de refaire des extractions d'une année sur l'autre quand des campagnes de dépôt massif sont organisées, par exemple.

Ouvrir le fichier et concaténer les différentes années (mettre "2022" dans le code quand les données de 2021 seront injectées) :

In [28]:
# Une année de début peut etre fournie, sinon par défaut commence en 2016
def read_hal(year=2016):
    hal_per_year = []
    for y in range (year, 2021):
        hal_df = pandas.read_csv("Data/raw/{}/hal_inserm_{}.csv".format(str(y), str(y)), sep="\t", encoding='latin-1')
        hal_per_year.append(hal_df)
    
    
    full_hal = pandas.concat(hal_per_year)
                     
    return full_hal

In [29]:
hal_df = read_hal()

In [30]:
hal_df

,doi
0,10.1016/j.rbmo.2016.03.005
1,10.1016/j.cytogfr.2016.07.003
2,10.1038/cddis.2016.88
3,10.1186/s12878-016-0048-6
4,10.1016/j.nmni.2016.04.005
...,...
4949,10.1038/s41467-019-14086-1
4950,10.1016/j.immuni.2020.07.021
4951,10.1371/journal.pntd.0008317
4952,10.7554/eLife.45047


 <span style="color:green"> **Les données de HAL sont à présent nettoyées et stockées dans la variable intitulée "hal_df".** </span>

## <span style="color:orange">Rassembler les 3 listes de DOI (WoS, PubMed, HAL) et enlever les doublons</span>

### Concaténer les 3 listes de données

 <span style="color:red"> **Pour adapter ce code à votre établissement, vous pouvez changer le nom de la variable "all_inserm" et y mettre à la place, par exemple, all_cote_azur. Mais attention dans ce cas à bien modifier le nom de la variable dans tout le reste du code.** </span>

In [31]:
all_inserm = pandas.concat([wos_df, pubmed_df, hal_df])

### Enlever les lignes sans DOI

In [32]:
import numpy

In [33]:
#mask_doi = all_inserm[column_name].notna()
#mask_doi

Le résultat correspond à la concaténation de toutes les lignes des 5 fichiers, y compris celles qui ne comprennent pas de DOI.

In [34]:
# doi_inserm_doublons = all_inserm[mask_doi]
doi_inserm_doublons = all_inserm.dropna()

In [35]:
doi_inserm_doublons

,doi
0,10.1186/s13601-016-0137-4
1,10.1371/journal.pone.0168349
2,10.3389/fnmol.2016.00160
3,10.1186/s12916-016-0763-7
4,10.3389/fimmu.2016.00632
...,...
4949,10.1038/s41467-019-14086-1
4950,10.1016/j.immuni.2020.07.021
4951,10.1371/journal.pntd.0008317
4952,10.7554/eLife.45047


Nous avons ici la liste de tous les DOI recensés dans nos 5 sources d'information. Mais attention, il peut y avoir des doublons : les DOI peuvent notamment être écrits en minuscules comme en majuscules.

### Enlever les doublons

Avant d'enlever les doublons, il faut normaliser les DOI et tous les passer en minuscules.

In [36]:
doi_inserm_doublons_minuscule = doi_inserm_doublons[column_name].str.lower()

In [37]:
doi_inserm_doublons_df = pandas.DataFrame({column_name : doi_inserm_doublons_minuscule})

Après avoir passé l'ensemble de la DataFrame en minuscules, on peut enlever les doublons.

In [38]:
doi_inserm_final = doi_inserm_doublons_df.drop_duplicates(ignore_index=True)

Réindexer la DataFrame pour que le nom des lignes soit propre :

In [39]:
# Utiliser plutot le parametre 'ignore_index=True' lors du drop_duplicates
# doi_inserm_final.reset_index(drop=True, inplace=True)

In [40]:
doi_inserm_final

,doi
0,10.1186/s13601-016-0137-4
1,10.1371/journal.pone.0168349
2,10.3389/fnmol.2016.00160
3,10.1186/s12916-016-0763-7
4,10.3389/fimmu.2016.00632
...,...
84592,10.1071/rdv32n2ab159
84593,10.1177/2048872619883997
84594,10.1051/0004-6361/201937297
84595,10.1111/ene.14521


**On obtient 21 457 DOI pour la période 2016-2020.**  <span style="color:red"> **Ce chiffre est à actualiser chaque année : dans le code, cela se fera automatiquement.** </span>

### Créer dans le dossier "outputs" un fichier CSV avec la liste des DOI

De même que précédemment, chaque établissement peut utiliser un nom différent : il faut simplement penser à remplacer "lorraine" par le nom de l'établissement à chaque fois qu'il apparaît dans le code et toujours l'écrire de la même façon.

In [41]:
doi_inserm_final.to_csv("Data\outputs\last_doi_inserm.csv",index=False)

Pour obtenir cette liste dans un format Excel classique, saisir le code suivant.

In [42]:
doi_inserm_final.to_excel("Data\outputs\last_doi_inserm.xlsx",index=False)

ModuleNotFoundError: No module named 'openpyxl'

### Générer fichier excel à partir du csv

In [45]:
import pandas

In [46]:
import csv

In [47]:
with open("Data/outputs/bso-publications-latest_180036048_enriched.csv") as f:
    inserm_str = f.read()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 5330: character maps to <undefined>

In [48]:
inserm_df = pandas.read_csv("Data/outputs/bso-publications-latest_180036048_enriched.csv",sep=",")

In [49]:
inserm_df

,doi,year,title,journal_issns,journal_issn_l,journal_name,publisher,publisher_dissemination,hal_id,pmid,...,bso_local_affiliations,is_oa,journal_is_in_doaj,journal_is_oa,observation_date,oa_host_type,oa_colors,licence_publisher,licence_repositories,repositories
0,10.1177/0962280213475643,2013.0,A graphical method to assess distribution assu...,"0962-2802,1477-0334",0962-2802,Statistical Methods in Medical Research,SAGE Publications,SAGE Publications,NaN,23427224.0,...,180036048;196917744,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN
1,10.1177/0962280213490014,2013.0,Sensitivity analysis of incomplete longitudina...,"0962-2802,1477-0334",0962-2802,Statistical Methods in Medical Research,SAGE Publications,SAGE Publications,NaN,23698867.0,...,180036048;197819444,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN
2,10.1177/0962280213489234,2016.0,Predictions in an illness-death model,"0962-2802,1477-0334",0962-2802,Statistical Methods in Medical Research,SAGE Publications,SAGE Publications,NaN,23698869.0,...,180036048,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN
3,10.1177/0962280213507034,2016.0,Propensity score estimators for the average tr...,"0962-2802,1477-0334",0962-2802,Statistical Methods in Medical Research,SAGE Publications,SAGE Publications,NaN,24201469.0,...,180036048,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN
4,10.1016/j.neuchi.2013.06.004,2016.0,Giant intracranial aneurysms in the paediatric...,0028-3770,0028-3770,Neurochirurgie,Elsevier BV,Elsevier,NaN,24210289.0,...,180036048,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84034,10.7554/elife.52665,2019.0,"Pathway-, layer- and cell-type-specific thalam...",2050-084X,2050-084X,eLife,"eLife Sciences Publications, Ltd","eLife Sciences Publications, Ltd",NaN,NaN,...,180036048,True,True,True,2021Q4,publisher;repository,green;gold,cc-by,cc-by,PubMed Central;infoscience.epfl.ch
84035,10.7754/clin.lab.2016.161008,2017.0,Point of Care Cardiac Troponin Assay Analytica...,1433-6510,1433-6510,Clinical Laboratory,Clinical Laboratory Publications,Clinical Laboratory Publications,NaN,NaN,...,180036048,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN
84036,10.7754/clin.lab.2018.181210,2019.0,An Extremely High IgA Value in a Young Child,1433-6510,1433-6510,Clinical Laboratory,Clinical Laboratory Publications,Clinical Laboratory Publications,NaN,NaN,...,180036048,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN
84037,10.7868/s0044467717050057,2017.0,ФОТОФАРМАКОЛОГИЯ: КРАТКИЙ ОБЗОР НА ПРИМЕРЕ УПР...,0044-4677,0044-4677,Журнал высшей нервной деятельности им И П Павлова,Akademizdatcenter Nauka,Akademizdatcenter Nauka,NaN,NaN,...,180036048,False,False,False,2021Q4,closed,closed,NaN,NaN,NaN


In [ ]:
inserm_df.to_excel("Data\outputs\last_inserm_20220302.xlsx",index=False)